# Explore Dataset - Homework exercice 1 (Programming task)

## Introduction

This dataset consists of tv shows and movies available on Netflix as of 2019. The dataset is collected from Flixable which is a third-party Netflix search engine.

Dataset with Netflix content available as of 2019
[Netflix content dataset](https://www.kaggle.com/shivamb/netflix-shows)

This dataset was gathered from kaggle and it was used to cluster movies togheter to find similar movies and create recommendations.

### Inspiration:

1. Understanding what content is available in different countries
2. Identifying similar content by matching text-based features
3. Network analysis of Actors / Directors and find interesting insights
4. Is Netflix has increasingly focusing on TV rather than movies in recent years.


In [ ]:
#Import Pandas library
import pandas as pd

#Import Numpy library
import numpy as np

#Import Plotly library
import plotly.express as px

## Load dataset and first analysis

In [ ]:
# Load dataset from the folder datasets
netflix_df = pd.read_csv("./datasets/netflix_titles.csv")
# returns rows
netflix_df.head()

In [ ]:
# Print a concise summary of a DataFrame
netflix_df.info()

In [ ]:
# Return a tuple representing the dimensionality of the DataFrame.
netflix_df.shape

In [ ]:
# Generate descriptive statistics.
netflix_df.describe(include=np.object)

In [ ]:
# Examine how often specific values occur in release_year column
netflix_df['release_year'].value_counts()

## Quering the dataset

In [ ]:
netflix_df[netflix_df['release_year'] > 2019]

## Sorting

In [ ]:
netflix_df.sort_values(by='release_year').head()

## Transforming Dataset

In [ ]:
# Convert season to minutes and convert x min to x (convert string to int)
def convert_duration(d):
    a = d.split(' ')
    if a[1] == 'min':
        return int(a[0])
    else:
        #estimate season duration
        # 12 = average number of episodes
        # 45 = average number of minutes per episode
        return int(a[0])*12*45
        
netflix_df['duration'] = netflix_df['duration'].apply(convert_duration)
netflix_df.head()

In [ ]:
# Count the number of Nulls
netflix_df.isnull().sum()

### Missing values

In [ ]:
# Remove Nulls
netflix_df.dropna(inplace=True)

In [ ]:
# Count the number of Nulls
netflix_df.isnull().sum()

### Convert String Date to Datetime

1. Conversion Times

In [ ]:
# Conversion times with inference and without

'''
%timeit pd.to_datetime(netflix_df['date_added'], infer_datetime_format=True)

%timeit pd.to_datetime(netflix_df['date_added'], infer_datetime_format=False)
'''

![Convertion Times](images/convertion_times.png)

2. Convert date to datetime

In [ ]:
netflix_df['date_added'] = pd.to_datetime(netflix_df['date_added'], infer_datetime_format=False)

In [ ]:
netflix_df.dropna(inplace=True)

In [ ]:
netflix_df.info()

## Visualizing

### Plot

In [ ]:
# Relase year plot (bar)
netflix_df["release_year"].value_counts().head(10).plot(kind="bar")

### Histogram

1. Release year histogram

In [ ]:
# Relase year Histogram
netflix_df.hist(column='release_year', bins =20)

2. Duration histogram

In [ ]:
netflix_df.hist(column='duration', bins=50)

## Top rated movies on netflix

Due to the lack of movie ratings on the original dataset, we decided to use the ratings available on IMDb by merging the original dataset with the ratings from the second.
[IMDb movies extensive dataset](https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset)

In [ ]:
#Loads IMDB ratings dataset
imdb_ratings=pd.read_csv('datasets/IMDb ratings.csv',usecols=['weighted_average_vote'])
imdb_titles=pd.read_csv('datasets/IMDb movies.csv', usecols=['title','year','genre'], dtype=str)

#Builds a dataframe containing movie's title, release year, rating and genre
ratings = pd.DataFrame({'Title':imdb_titles.title,
                    'Release Year':imdb_titles.year,
                    'Rating': imdb_ratings.weighted_average_vote,
                    'Genre':imdb_titles.genre})

#Removes duplicate values from the dataframe and rebuilds it
ratings.drop_duplicates(subset=['Title','Release Year','Rating'], inplace=True)
ratings.shape

In [ ]:
imdb_titles.info()

In [ ]:
imdb_ratings.info()

In [ ]:
#Removes rows with empty values from the dataframe and rebuilds it
ratings.dropna(inplace=True)

#Merges the two dataframes into a new one
joint_data=ratings.merge(netflix_df,left_on='Title',right_on='title',how='inner')

#Sorts the dataframe into descending rating order
joint_data=joint_data.sort_values(by='Rating', ascending=False)

In [ ]:
joint_data.info()

In [ ]:
#Picks the first 10 elements from the dataframe and creates a list
top_rated=joint_data[0:10]

#Builds sunburst visualization with Plotly
fig = px.sunburst(
    top_rated,
    path=['title','country'],
    values='Rating',
    color='Rating')
fig.show()

## Top Movie Makers

In [ ]:
#Creates a dataframe based on the value counts of movies by 'Country' column
country_count=joint_data['country'].value_counts().sort_values(ascending=False)
country_count=pd.DataFrame(country_count)

#Picks 11 top counts
topcountries=country_count[0:10]
topcountries

In [ ]:
#Get countries to list format
topcountries.to_dict()['country'].keys()

In [ ]:
#Get movie counts to list format
topcountries['country'].values.tolist()

In [ ]:
#Converts data to dictionary to fit plotting criteria
data = dict(
    number=topcountries['country'].values.tolist(),
    country=topcountries.to_dict()['country'].keys())

fig = px.histogram(data, x='country', y='number')
fig.show()

#Plots funnel image
fig = px.funnel(data, x='number', y='country')
fig.show()

## Average IMDb Movie rating on Netflix

In [ ]:
np.floor(joint_data['Rating']).value_counts()

In [ ]:
fig = px.bar(np.floor(joint_data['Rating']).value_counts())
fig.show()

In [ ]:
fig = px.histogram(joint_data['rating'])
fig.show()